# Review of Day 1

During Day 1, we introduced Julia basics, and saw that Julia has the following features:

- Familiar syntax
- Easy to make generic code ("write once, run everywhere")
- Julia is fast

Day 2 will be about Performance and Parallelism.
We will see how to profile code, some performance gotchas, and several ways to parallelize code using Julia.

# Simulations: profiling and performance
## Random walks

In this notebook, we will look at one of the simplest types of Monte Carlo numerical simulation, random walks.

In the simplest random walk, a particle starts at $0$ and jumps to the left ($-1$) or the right ($+1$) with equal probability.

The following is a simple implementation of a single random walk:

In [1]:
numsteps = 1000
pos = 0 

for j in 1:numsteps
            
    if rand() < 0.5
        step = -1
    else
        step = +1
    end
            
    pos += step 
    
end

pos

22

In [2]:
using Plots; gr()

Plots.GRBackend()

The code seems to execute almost instantaneously, but we should **profile** (time) it:

In [59]:
memory = @allocated rand(1000)

8128

In [60]:
memory

8128

In [61]:
using Interact

In [63]:
exponents = 3:7
values = [10^k for k in exponents]

5-element Array{Int64,1}:
     1000
    10000
   100000
  1000000
 10000000

In [64]:
@manipulate for k in exponents
    @allocated rand(10^k)
end

Interact.Slider{Int64}(Signal{Int64}(5, nactions=0),"k",5,3:7,"d",true)

800096

You might expect exactly $8*(10^k)$, but there is a bit of overhead in a Julia array (80 bytes?)

In [56]:
@time begin
    
    numsteps = 1000
    pos = 0 
    for j in 1:numsteps

        if rand() < 0.5
            step = -1
        else
            step = +1
        end

        pos += step 
    end
    
end

  0.000186 seconds (1.98 k allocations: 46.578 KB)


In [65]:
@elapsed sin(10)

0.00406933

In [66]:
@allocated sin(10)

0

Although it's fast, it seems to be allocating memory unexpectedly. This is a **warning** of a possible **type instability**.

Let's wrap it in a function, which is good programming practice, and allows us to have `numsteps` as a paramater.
It turns out to have an additional, important effect in Julia.

In [1]:
"""Single 1D random walk from the origin.
Returns the final position after `numsteps` steps."""
function walk(numsteps=1000)  # default value of the parameter
    
    pos = 0 
    
    for j in 1:numsteps

        if rand() < 0.5   # can replace by rand(Bool)
            step = -1
        else
            step = +1
        end

        pos += step 
    end
    
    return pos
    
end

walk

In [68]:
@time walk(1)

  0.007574 seconds (2.86 k allocations: 130.777 KB)


16

In [69]:
@time walk(1000)

  0.000011 seconds (4 allocations: 160 bytes)


-12

In [70]:
@time walk(1000)

  0.000011 seconds (4 allocations: 160 bytes)


-46

In [74]:
@manipulate for k in 3:10
    log10(@elapsed walk(10^k))
end

Interact.Slider{Int64}(Signal{Int64}(6, nactions=0),"k",6,3:10,"d",true)

-2.11911269820074

In [81]:
rand(1:10)

2

In [82]:
a = true
typeof(a)

Bool

In [92]:
rand(Bool)

false

In [93]:
@which rand(Bool)

rand(T::Type) at random.jl:228

In [94]:
@edit rand(Bool)

In [96]:
ENV["EDITOR"] = "vim"

"vim"

In [ ]:
@edit rand(Bool)

Vim: Warning: Output is not to a terminal
Vim: Warning: Input is not from a terminal




























[O]pen Read-Only, (E)dit anyway, (R)ecover, (D)elete it, (Q)uit, (A)bort: 

In [95]:
ENV

Base.EnvHash with 35 entries:
  "FONTCONFIG_PATH"            => "/Applications/Julia-0.5.app/Contents/Resourc…
  "TERM_PROGRAM_VERSION"       => "3.0.12"
  "TMPDIR"                     => "/var/folders/h0/68t1xc991mq20mrkn6yy6n5h0000…
  "LOGNAME"                    => "dpsanders"
  "USER"                       => "dpsanders"
  "HOME"                       => "/Users/dpsanders"
  "PATH"                       => "/Applications/Julia-0.5.app/Contents/Resourc…
  "CLICOLOR"                   => "1"
  "DISPLAY"                    => "/private/tmp/com.apple.launchd.I4LZaH4k5G/or…
  "TERM_PROGRAM"               => "iTerm.app"
  "LANG"                       => "en_US.UTF-8"
  "TK_LIBRARY"                 => "/System/Library/Frameworks/Tk.framework/Vers…
  "TERM"                       => "xterm-256color"
  "SHELL"                      => "/bin/bash"
  "COLORFGBG"                  => "7;0"
  "SHLVL"                      => "1"
  "XPC_FLAGS"                  => "0x0"
  "ITERM_SESSION_ID"          

Messages: 
- Wrap everything in a function
- Run the function once, before timing only on the *second* run

## Interlude on vectors: collecting data

Now we run it several times to collect data:

In [2]:
walk()

-24

In [5]:
numsteps = 10000
data = [walk(numsteps) for i in 1:10]

10-element Array{Int64,1}:
  120
 -152
 -112
   16
  138
   74
  198
  -96
  -74
   58

Another possibility:

In [6]:
[ ]

0-element Array{Any,1}

In [9]:
Int[ ]   # better for performance

0-element Array{Int64,1}

In [10]:
[ 0 ]

1-element Array{Int64,1}:
 0

In [12]:
[ 0, 1.0 ]  # it promotes everything to a common type

2-element Array{Float64,1}:
 0.0
 1.0

In [14]:
v = [ 0, 1.0, "hello"]

3-element Array{Any,1}:
 0       
 1.0     
  "hello"

In [15]:
push!(v, false)

4-element Array{Any,1}:
     0       
     1.0     
      "hello"
 false       

In [16]:
v

4-element Array{Any,1}:
     0       
     1.0     
      "hello"
 false       

The "bang" (`!`) is a convention that means that the function **modifies** its (first) argument.

In [18]:
data = Int[ ]   # empty array of `Int`s

0-element Array{Int64,1}

In [20]:
x = 3

3

In [21]:
show(x)

3

In [22]:
@show x

x = 3


3

In [23]:
data = Int[ ]   # empty array of `Int`s

numwalkers = 10

for i in 1:numwalkers
    pos = walk()
    push!(data, pos)
    @show data
end

data

data = [2]
data = [2,78]
data = [2,78,-26]
data = [2,78,-26,14]
data = [2,78,-26,14,-8]
data = [2,78,-26,14,-8,40]
data = [2,78,-26,14,-8,40,54]
data = [2,78,-26,14,-8,40,54,14]
data = [2,78,-26,14,-8,40,54,14,-10]
data = [2,78,-26,14,-8,40,54,14,-10,-58]


10-element Array{Int64,1}:
   2
  78
 -26
  14
  -8
  40
  54
  14
 -10
 -58

Alternative: pre-allocate the array

In [24]:
numwalkers = 10

data = zeros(numwalkers)

10-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [25]:
data = zeros(Int, numwalkers)

10-element Array{Int64,1}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [26]:
data = Vector{Int}(numwalkers)  # fills with random garbage; cf. malloc

10-element Array{Int64,1}:
    4726803984
            13
    4647731232
    4728607248
             0
    4726933952
    4726934032
    4726804016
    4726934112
 1099528470529

In [27]:
data = Vector{Int}(numwalkers)  # fills with random garbage; cf. malloc

for i in 1:numwalkers
    pos = walk()
    data[i] = pos  # this does **not** automatically grow the array (as it would in Matlab)
    @show data
end

data

data = [44,13,4647731232,4728608784,0,4727617776,4727617856,4727334832,4727617936,1099528470529]
data = [44,-12,4647731232,4728608784,0,4727617776,4727617856,4727334832,4727617936,1099528470529]
data = [44,-12,-2,4728608784,0,4727617776,4727617856,4727334832,4727617936,1099528470529]
data = [44,-12,-2,-62,0,4727617776,4727617856,4727334832,4727617936,1099528470529]
data = [44,-12,-2,-62,34,4727617776,4727617856,4727334832,4727617936,1099528470529]
data = [44,-12,-2,-62,34,-24,4727617856,4727334832,4727617936,1099528470529]
data = [44,-12,-2,-62,34,-24,42,4727334832,4727617936,1099528470529]
data = [44,-12,-2,-62,34,-24,42,14,4727617936,1099528470529]
data = [44,-12,-2,-62,34,-24,42,14,-36,1099528470529]
data = [44,-12,-2,-62,34,-24,42,14,-36,22]


10-element Array{Int64,1}:
  44
 -12
  -2
 -62
  34
 -24
  42
  14
 -36
  22

In [28]:
data = Int[] # fills with random garbage; cf. malloc

for i in 1:numwalkers
    pos = walk()
    data = [data; pos]  # common Matlabism
    @show data
end

data

data = [44]
data = [44,30]
data = [44,30,-30]
data = [44,30,-30,48]
data = [44,30,-30,48,24]
data = [44,30,-30,48,24,-18]
data = [44,30,-30,48,24,-18,56]
data = [44,30,-30,48,24,-18,56,18]
data = [44,30,-30,48,24,-18,56,18,4]
data = [44,30,-30,48,24,-18,56,18,4,4]


10-element Array{Int64,1}:
  44
  30
 -30
  48
  24
 -18
  56
  18
   4
   4

In [29]:
function method1(N)  # Matlabism
    data = Int[] 

    for i in 1:N
        pos = walk()
        data = [data; pos]  
    end
    
    data  # will return data
end

method1 (generic function with 1 method)

In [30]:
function method2(N)  # push!
    data = Int[ ]   

    for i in 1:N
        pos = walk()
        push!(data, pos)
    end

    data
end

method2 (generic function with 1 method)

In [31]:
function method3(N)  # preallocate
    data = zeros(Int, N)  

    for i in 1:N
        pos = walk()
        data[i] = pos
    end

    data
end

method3 (generic function with 1 method)

First thing to do when profiling in Julia: run the functions once each

In [32]:
method1(1)
method2(1)
method3(1)

1-element Array{Int64,1}:
 -36

In [34]:
N = 100000
@time method1(N)
@time method2(N)
@time method3(N)

 18.783738 seconds (5.89 M allocations: 37.447 GB, 12.79% gc time)
  0.804275 seconds (21 allocations: 2.001 MB, 0.23% gc time)
  0.780113 seconds (6 allocations: 781.484 KB)


100000-element Array{Int64,1}:
 -18
  56
  20
  24
 -56
 -20
  42
 -22
  16
 -10
  16
 -32
  18
   ⋮
 -10
  -8
  -4
 -34
  36
 -52
  44
  -6
  30
  52
  16
 -28

The correct way to profile is with `@benchmark` from `BenchmarkTools.jl`

In [37]:
using BenchmarkTools

N = 10000
display(@benchmark method1(N))
display(@benchmark method2(N))
display(@benchmark method3(N))

BenchmarkTools.Trial: 
  memory estimate:  401.29 mb
  allocs estimate:  581300
  --------------
  minimum time:     380.563 ms (12.89% GC)
  median time:      423.159 ms (12.63% GC)
  mean time:        455.725 ms (15.49% GC)
  maximum time:     669.648 ms (29.66% GC)
  --------------
  samples:          11
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

BenchmarkTools.Trial: 
  memory estimate:  256.70 kb
  allocs estimate:  14
  --------------
  minimum time:     74.918 ms (0.00% GC)
  median time:      79.899 ms (0.00% GC)
  mean time:        79.730 ms (0.00% GC)
  maximum time:     84.844 ms (0.00% GC)
  --------------
  samples:          63
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

BenchmarkTools.Trial: 
  memory estimate:  78.20 kb
  allocs estimate:  2
  --------------
  minimum time:     73.927 ms (0.00% GC)
  median time:      78.467 ms (0.00% GC)
  mean time:        79.268 ms (0.00% GC)
  maximum time:     95.222 ms (0.00% GC)
  --------------
  samples:          64
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

## Collecting the random walk data

In [39]:
numsteps   = 1000
numwalkers = 10000

@time data = [walk(numsteps) for i in 1:numwalkers]  # final positions

  0.101981 seconds (12.15 k allocations: 621.639 KB)


10000-element Array{Int64,1}:
 -10
   8
 -52
 -36
   2
  -4
  34
   2
  32
 -14
 -38
  36
 -22
   ⋮
 -34
 -40
  28
 -10
  38
  12
  -2
  -2
  42
  18
  50
  18

### Let's do statistics

A population of simple random walks should have mean $0$ and variance equal to the total time. Let's check it:

In [44]:
mean(data)  # mean should be 0 for simple (unbiased) random walks

-0.2056

In [46]:
var(data), numsteps  # variance should be exactly the number of steps

(1020.6405926992701,1000)

In [49]:
≈(var(data), numsteps, rtol=3e-2)  # relative tolerance

true

We can plot the histogram:

In [50]:
using Plots; gr()

histogram(data, nbins=100)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -100 
 
 
 0 
 
 
 100 
 
 
 0 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

Again, however, the high number of allocations is suspect -- in Julia, this is usually a warning that there is a "type instability". We again try wrapping it in a function, even though it seems so simple:

In [40]:
function run_walks(numwalkers, numsteps)

    data = [walk(numsteps) for i in 1:numwalkers]

    return data
end

run_walks (generic function with 1 method)

In [42]:
numsteps   = 1000
numwalkers = 10000

data = run_walks(1, 1)  # compile the function

# now profile it:
@time data = run_walks(numwalkers, numsteps);  

  0.077005 seconds (7 allocations: 78.375 KB)


Summary:
- Simple simulation
- Put everything in a function
- Collect data using comprehensions or `push!`
- Use `@time` (on the second run) or `@benchmark` to profile

# DistributedArrays 

Jump to [notebook 1a.](1a. Basics of distributed arrays.ipynb) for the basics of distributed arrays in Julia.

# Basic parallelism

In [16]:
addprocs(2)

2-element Array{Int64,1}:
 2
 3

In [17]:
@everywhere using DistributedArrays

In [18]:
@everywhere function walk(numsteps)
    pos = 0

    for j in 1:numsteps
        
        if rand(Bool)  # NB
            step = -1
        else
            step = +1
        end
        
        pos += step # ifelse(rand() < 0.5, -1, +1)
    end
    
    return pos
end

In serial:

In [19]:
data = [walk(numsteps) for i in 1:numwalkers] 

LoadError: UndefVarError: numwalkers not defined

In [ ]:
walkers = distribute(1:numwalkers);

In [ ]:
walkers.indexes

In [23]:
@everywhere begin
    numsteps   = 10000
    numwalkers = 100000 
end

walkers = distribute(1:numwalkers)

100000-element DistributedArrays.DArray{Int64,1,UnitRange{Int64}}:
      1
      2
      3
      4
      5
      6
      7
      8
      9
     10
     11
     12
     13
      ⋮
  99989
  99990
  99991
  99992
  99993
  99994
  99995
  99996
  99997
  99998
  99999
 100000

In [24]:
walkers.indexes

2-element Array{Tuple{UnitRange{Int64}},1}:
 (1:50000,)     
 (50001:100000,)

A function that returns a constant:


In [26]:
f() = 3  # no arguments

f (generic function with 1 method)

An anonymous function that takes one argument, that it ignores, and returns a constant:

In [27]:
_ -> 3

(::#45) (generic function with 1 method)

An anonymous function that returns something random:

In [29]:
g = _ -> walk(numsteps)

(::#49) (generic function with 1 method)

In [31]:
g(1)

-160

In [32]:
g("string")

-12

In [34]:
h = _ -> rand()
map(h, 1:5)

5-element Array{Float64,1}:
 0.838203
 0.854722
 0.74231 
 0.111471
 0.630536

In [45]:
map(rand, 1:5)

5-element Array{Array{Float64,1},1}:
 [0.139488]                                    
 [0.539704,0.0370874]                          
 [0.821382,0.145536,0.265507]                  
 [0.102901,0.391612,0.343755,0.815076]         
 [0.72609,0.731211,0.107925,0.867699,0.0608978]

In [46]:
[rand(1), rand(2), rand(3), rand(4), rand(5)]

5-element Array{Array{Float64,1},1}:
 [0.00147708]                                  
 [0.99099,0.961199]                            
 [0.376861,0.26289,0.876219]                   
 [0.240896,0.696245,0.674086,0.925543]         
 [0.241501,0.775125,0.268784,0.620798,0.296621]

In [47]:
rand(3)

3-element Array{Float64,1}:
 0.568287
 0.342367
 0.975112

In [48]:
map(_->rand(), 1:5)

5-element Array{Float64,1}:
 0.515412
 0.929861
 0.978093
 0.815971
 0.622732

In [51]:
map(rand, ones(Int, 5))

5-element Array{Array{Float64,1},1}:
 [0.742302] 
 [0.460336] 
 [0.0283694]
 [0.693124] 
 [0.608195] 

In [52]:
rand.(1:5)

5-element Array{Array{Float64,1},1}:
 [0.332193]                                    
 [0.572766,0.919379]                           
 [0.549278,0.804477,0.974742]                  
 [0.986861,0.729855,0.438648,0.858234]         
 [0.101719,0.957467,0.215089,0.916968,0.363882]

In [50]:
ones(5)

5-element Array{Float64,1}:
 1.0
 1.0
 1.0
 1.0
 1.0

In [35]:
[h(1), h(2), h(3), h(4), h(5)]

5-element Array{Float64,1}:
 0.911449
 0.704877
 0.542176
 0.810614
 0.755413

In [36]:
gg = _ -> walk(numsteps)

(::#55) (generic function with 1 method)

In [37]:
[gg(1), gg(2), gg(3)]

3-element Array{Int64,1}:
 152
  50
 -38

In [38]:
map(gg, [1,2,3])  # or map(gg, 1:3)

3-element Array{Int64,1}:
 -90
 -20
  84

In [39]:
typeof(walkers)

DistributedArrays.DArray{Int64,1,UnitRange{Int64}}

In [41]:
@time positions = map( _ -> walk(numsteps), walkers)

  3.601156 seconds (12.25 k allocations: 528.826 KB)


100000-element DistributedArrays.DArray{Int64,1,Array{Int64,1}}:
  -26
   64
 -168
   -6
   82
   40
  -80
 -136
  -58
  -80
   -6
 -268
  -44
    ⋮
 -134
    2
  -24
  226
  -42
  -82
   76
 -194
 -144
  178
    2
  -38

In [42]:
function run_serial(numwalkers, numsteps)
    data = map(_ -> walk(numsteps), 1:numwalkers)
end

run_serial (generic function with 1 method)

In [43]:
run_serial(1, 1)

1-element Array{Int64,1}:
 -1

In [44]:
@time run_serial(numwalkers, numsteps)

  6.950400 seconds (7 allocations: 781.500 KB)


100000-element Array{Int64,1}:
   32
  -54
 -118
    2
   14
  184
   34
   10
  -48
   -6
  274
   62
  258
    ⋮
  -64
    0
  172
   10
  -52
  104
  -14
  -46
  -78
  -42
   44
 -170

We do see almost 2x speedup with 2 processes!

In [56]:
positions

100000-element DistributedArrays.DArray{Int64,1,Array{Int64,1}}:
  -26
   64
 -168
   -6
   82
   40
  -80
 -136
  -58
  -80
   -6
 -268
  -44
    ⋮
 -134
    2
  -24
  226
  -42
  -82
   76
 -194
 -144
  178
    2
  -38

In [60]:
v = [1, -2, 3]
v .> 0

3-element BitArray{1}:
  true
 false
  true

In [61]:
v[v .> 0] = 10  # modify v using a bit-mask
# "modify v *where* v > 0

10

In [62]:
v

3-element Array{Int64,1}:
 10
 -2
 10

In [63]:
M = rand(5, 5)

5×5 Array{Float64,2}:
 0.771339   0.228275   0.868034  0.141481  0.504641
 0.838896   0.11143    0.756968  0.160655  0.607094
 0.103146   0.0674581  0.768882  0.69896   0.207889
 0.0874783  0.384836   0.989942  0.389488  0.740674
 0.28365    0.261405   0.770079  0.267742  0.755318

In [64]:
M .> 0.5

5×5 BitArray{2}:
  true  false  true  false   true
  true  false  true  false   true
 false  false  true   true  false
 false  false  true  false   true
 false  false  true  false   true

In [65]:
M[M .> 0.5] = 10

10

In [66]:
M

5×5 Array{Float64,2}:
 10.0        0.228275   10.0   0.141481  10.0     
 10.0        0.11143    10.0   0.160655  10.0     
  0.103146   0.0674581  10.0  10.0        0.207889
  0.0874783  0.384836   10.0   0.389488  10.0     
  0.28365    0.261405   10.0   0.267742  10.0     

In [67]:
mapreduce(x -> x > 0, +, positions) 

49684

In [70]:
+(false, true)  # inteprets them as numbers

1

In [71]:
@which +(false, true)

+(x::Bool, y::Bool) at bool.jl:48

Cf. random behaviour in financial markets.

In [68]:
map(x -> x > 0, positions)

100000-element DistributedArrays.DArray{Bool,1,Array{Bool,1}}:
 false
  true
 false
 false
  true
  true
 false
 false
 false
 false
 false
 false
 false
     ⋮
 false
  true
 false
  true
 false
 false
  true
 false
 false
  true
  true
 false

In [73]:
@which map(x -> x > 0, positions)

map(f, d::DistributedArrays.DArray) at /Users/dpsanders/.julia/v0.5/DistributedArrays/src/mapreduce.jl:6

In [59]:
positions .> 0  # terribly slow

49684

In [74]:
@which positions .> 0

.>(x, y) at operators.jl:69

In [75]:
@which 0 .< positions

.<(A, B::AbstractArray) at broadcast.jl:370

This was terribly slow since uses generic fallback, instead of a special method for `DArrays`.

Let's fix that:

In [77]:
import Base: .<
.<(A, B::DArray) = map(x -> x > A, B)

.< (generic function with 12 methods)

Now I should go and make a Pull Request to the [DistributedArrays.jl](https://github.com/JuliaParallel/DistributedArrays.jl) package

In [78]:
0 .< positions

100000-element DistributedArrays.DArray{Bool,1,Array{Bool,1}}:
 false
  true
 false
 false
  true
  true
 false
 false
 false
 false
 false
 false
 false
     ⋮
 false
  true
 false
  true
 false
 false
  true
 false
 false
  true
  true
 false

In [57]:
mean(positions)

-0.04856

In [58]:
var(positions)

10045.41121603856

In [ ]:
squared_positions = map(x->x^2, positions);

In [ ]:
mean(squared_positions) ≈ var(positions)

# Another example: random matrices

In [53]:
using Plots; gr()

Plots.GRBackend()

In [54]:
workers()

2-element Array{Int64,1}:
 2
 3

In [ ]:
# addprocs(4)

In [55]:
@everywhere begin
    using DistributedArrays
    using StatsBase
    using Plots
end

In [ ]:
@everywhere function stochastic(β = 2, n = 200)
    h = n ^ -(1/3)
    x = 0:h:10
    N = length(x)
    d = (-2 / h^2 .- x) + 2*sqrt(h*β) * randn(N) # diagonal
    e = ones(N - 1) / h^2                     # subdiagonal
  
    eigvals(SymTridiagonal(d, e))[N]        # smallest negative eigenvalue
end

Serial version:

In [ ]:
println("Serial version")

t = 10000
p = plot()
for β = [1,2,4,10,20]
    
    z = fit(Histogram, [stochastic(β) for i = 1:t], -4:0.01:1).weights
    plot!(midpoints(-4:0.01:1), z / sum(z) / 0.01)
end
p

A related parallel construct: `@parallel`. This does a "reduce" operation.

In [ ]:
println("@parallel version")

@everywhere t = 10000

p = plot()

for β = [1,2,4,10,20]
    
    z = @parallel (+) for p = 1:nprocs()
        fit(Histogram, [stochastic(β) for i = 1:t], -4:0.01:1).weights
    end
    
    plot!(midpoints(-4:0.01:1), z / sum(z) / 0.01)
end

p

In [ ]:
function dhist(x; closed=:left, nbins=10)
    
    hist_parts = DArray(p->fit(Histogram, localpart(x), closed=closed, nbins=nbins).weights, (nbins*length(x.pids),))
    
    reduce(+, map(pid -> @fetchfrom(pid, localpart(hist_parts)), hist_parts.pids))
      
end

In [ ]:
a = randn(10000)
d = distribute(a)

dhist(d)

## Draw a random walk

In [3]:
using Plots; gr()

Plots.GRBackend()

In [4]:
function trajectory(numsteps=1000)

    pos = 0 
    positions = [pos]

    for j in 1:numsteps

        if rand() < 0.5
            step = -1
        else
            step = +1
        end

        pos += step 
        push!(positions, pos)

    end
    
    positions
end


trajectory (generic function with 2 methods)

In [7]:
traj = trajectory(100)

101-element Array{Int64,1}:
  0
  1
  0
 -1
 -2
 -1
  0
  1
  2
  3
  2
  1
  0
  ⋮
  1
  0
  1
  0
  1
  0
 -1
 -2
 -3
 -2
 -1
 -2

In [10]:

traj = trajectory(1000)
plot()
hline!([0], ls=:dash)


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200 
 
 
 400 
 
 
 600 
 
 
 800 
 
 
 1000 
 
 
 -30 
 
 
 -20 
 
 
 -10 
 
 
 0 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 30.6037,15.4829 31.1691,3.93701 31.7346,15.4829 32.3001,3.93701 32.8655,15.4829 33.431,3.93701 33.9964,15.4829 34.5619,27.0288 35.1273,15.4829 35.6928,3.93701 
 36.2583,15.4829 36.8237,27.0288 37.3892,38.5747 37.9546,27.0288 38.5201,38.5747 39.0856,50.1206 39.651,38.5747 40.2165,27.0288 40.7819,38.5747 41.3474,27.0288 
 41.9129,38.5747 42.4783,50.1206 43.0438,61.6665 43.6092,73.2124 44.1747,84.7583 44.7402,73.2124 45.3056,84.7583 45.8711,96.3042 46.4365,107.85 47.002,119.396 
 47.5675,107.85 48.1329,96.3042 48.6984,107.85 49.2638,96.3042 49.8293,84.7583 50.3948,96.3042 50.9602,107.85 51.5257,96.3042 52.0911,84.7583 52.6566,96.3042 
 53.222,107.85 53.7875,119.396 54.353,130.942 54.9184,142.488 55.4839,130.942 56.0493,119.396 56.6148,130.942 57.1803,142.488 57.7457,154.034 58.3112,142.488 
 58.8766,154.034 59.4421,142.488 60.0076,154.034 60.573,165.58 61.1385,177.126 61.7039,165.58 62.2694,154.034 62.8349,165.58 63.4003,177.126 63.9658,188.671 
 64.5312,177.126 65.0967,188.671 65.6622,200.217 66.2276,211.763 66.7931,223.309 67.3585,211.763 67.924,223.309 68.4894,234.855 69.0549,246.401 69.6204,257.947 
 70.1858,269.493 70.7513,257.947 71.3167,269.493 71.8822,257.947 72.4477,246.401 73.0131,257.947 73.5786,269.493 74.144,281.039 74.7095,269.493 75.275,281.039 
 75.8404,292.585 76.4059,281.039 76.9713,292.585 77.5368,281.039 78.1023,292.585 78.6677,304.131 79.2332,292.585 79.7986,281.039 80.3641,292.585 80.9296,281.039 
 81.495,269.493 82.0605,281.039 82.6259,292.585 83.1914,281.039 83.7569,269.493 84.3223,281.039 84.8878,292.585 85.4532,281.039 86.0187,292.585 86.5841,304.131 
 87.1496,292.585 87.7151,281.039 88.2805,292.585 88.846,281.039 89.4114,269.493 89.9769,257.947 90.5424,246.401 91.1078,234.855 91.6733,223.309 92.2387,234.855 
 92.8042,246.401 93.3697,257.947 93.9351,269.493 94.5006,257.947 95.066,269.493 95.6315,257.947 96.197,246.401 96.7624,257.947 97.3279,246.401 97.8933,234.855 
 98.4588,246.401 99.0243,257.947 99.5897,269.493 100.155,281.039 100.721,269.493 101.286,281.039 101.852,269.493 102.417,257.947 102.982,246.401 103.548,234.855 
 104.113,246.401 104.679,234.855 105.244,246.401 105.81,234.855 106.375,246.401 106.941,234.855 107.506,223.309 108.072,234.855 108.637,223.309 109.203,211.763 
 109.768,200.217 110.333,211.763 110.899,223.309 111.464,211.763 112.03,200.217 112.595,211.763 113.161,223.309 113.726,211.763 114.292,200.217 114.857,188.671 
 115.423,200.217 115.988,188.671 116.553,200.217 117.119,188.671 117.684,177.126 118.25,165.58 118.815,154.034 119.381,165.58 119.946,177.126 120.512,188.671 
 121.077,177.126 121.643,165.58 122.208,154.034 122.774,142.488 123.339,130.942 123.904,119.396 124.47,130.942 125.035,119.396 125.601,130.942 126.166,119.396 
 126.732,107.85 127.297,96.3042 127.863,84.7583 128.428,73.2124 128.994,84.7583 129.559,96.3042 130.125,84.7583 130.69,96.3042 131.255,107.85 131.821,119.396 
 132.386,107.85 132.952,96.3042 133.517,107.85 134.083,96.3042 134.648,84.7583 135.214,96.3042 135.779,107.85 136.345,96.3042 136.91,84.7583 137.475,73.2124 
 138.041,84.7583 138.606,73.2124 139.172,61.6665 139.737,50.1206 140.303,38.5747 140.868,27.0288 141.434,15.4829 141.999,3.93701 142.565,15.4829 143.13,27.0288 
 143.696,38.5747 144.261,50.1206 144.826,38.5747 145.392,50.1206 145.957,61.6665 146.523,73.2124 147.088,61.6665 147.654,73.2124 148.219,84.7583 148.785,96.3042 
 149.35,84.7583 149.916,96.3042 150.481,84.7583 151.047,73.2124 151.612,61.6665 152.177,50.1206 152.743,61.6665 153.308,50.1206 153.874,38.5747 154.439,27.0288 
 155.005,15.4829 155.57,3.93701 156.136,15.4829 156.701,3.93701 157.267,15.4829 157.832,27.0288 158.397,38.5747 158.963,27.0288 159.528,38.5747 160.094,27.

In [15]:
p = plot(legend=false)

for i in 1:n
    traj = trajectory(100)
    plot!(traj)
end

hline!([0], ls=:dash)

p

LoadError: MethodError: no method matching @gif(::Symbol, ::Expr)[0m
Closest candidates are:
  @gif([1m[31m::Expr[0m, ::Any...) at /Users/dpsanders/.julia/v0.5/Plots/src/animation.jl:172[0m